In [ ]:
from pywinauto.application import Application
import pywinauto
from pywinauto import *
from time import sleep
from tqdm import tqdm
from pywinauto.timings import wait_until, wait_until_passes

In [ ]:
def auto_review_new():
    sleep(2)
    app = Application(backend='uia').connect(best_match='Kofax Transformation Modules - Document Review')
    app_dlg = app.window(best_match='Kofax Transformation Modules - Document Review')
    
    # check warning batch contain rejected pages
    sendqc_dlg = app_dlg.descendants(title="The batch contains either rejected documents or rejected pages. Send it to Quality Control?")
    if(len(sendqc_dlg)>0):
        # send it to qc batch
        sendqc_yes = app_dlg.descendants(title="Yes", control_type="Button")
        sendqc_yes[0].type_keys('{ENTER}')
        return None
    
    # check warning batch complete
    complete_dlg = app_dlg.descendants(title="All problems in the batch have been fixed. Close batch?")

    if(len(complete_dlg)>0):
        # close batch
        warning_close_batch_ok = app_dlg.descendants(title="OK", control_type="Button")
        warning_close_batch_ok[0].type_keys('{ENTER}')
        return None
    
    # check no problem in batch
    noproblem_dlg = app_dlg.descendants(title="There are no problems in the current batch. Close batch?")
    if(len(noproblem_dlg)>0):
        # close batch
        noproblem_ok = app_dlg.descendants(title="Yes", control_type="Button")
        noproblem_ok[0].type_keys('{ENTER}')
        return None
    
    # check no documents to process
    nodoc_dlg = app_dlg.descendants(title="No documents to process, batch will be closed.")
    if(len(nodoc_dlg)>0):
        # close batch
        nodoc_ok = app_dlg.descendants(title="OK", control_type="Button")
        nodoc_ok[0].type_keys('{ENTER}')
        return None
    
    # define image trees
    tree_view = app_dlg.TreeView
    trees = tree_view.children(title="Kofax.DocumentReview.DocReviewNode")
    
    # define to split
    to_split = []

    for tree in trees:
        # identifikasi page dengan error
        error_images = len(tree.children()[0].children(class_name='ListBox')[0].children())
        if(error_images>1):
            to_split.append(tree)
    
    if(len(to_split)>0):
        for i in to_split:
            i.select()
            images = i.children()[0].children(class_name='ListBox')[0].children()
            for j in range(len(images)-1, 0, -1):
                images[j].select()
                images[j].click_input()
                sleep(2)
                # split dokumen Ctrl+T
                images[j].type_keys('^T')
    else:
        print('No Split')
    
    # define image trees
    tree_view = app_dlg.TreeView
    trees = tree_view.children(title="Kofax.DocumentReview.DocReviewNode")
    
    # untuk dokumen (page > 2)
    if len(trees)>2:
        for i in range(0, len(trees)):
            # identifikasi page dengan error
            error_images = len(trees[i].children()[0].children(class_name='ListBox')[0].children())
            
            if(error_images>0):
                combobox = trees[i].children()[0].children(class_name = 'ComboBox')
                
                # check combobox is visible
                trees[i].select()
                image_0 = trees[i].children()[0].children(class_name='ListBox')[0].children()[0]
                image_0.select()
                image_0.click_input()
                
                if(not combobox[0].children()[0].is_visible()):
                    combobox[0].click_input()

                # expand combobox
                combobox[0].children()[0].type_keys("%{DOWN}")
                combobox[0].expand()
                sleep(1)

                # klasifikasi jenis dokumen
                if(i>1):
                    print('image_ke_{} classified as : SP2020_C1_ART'.format(i+1))
                    try:
                        list_items = combobox[0].children(control_type="ListItem")
                        if(list_items[0].window_text()=='SP2020_C1_ART'):
                            list_items[0].select()
                        else:
                            list_items[1].select()
                    except :
                        break
                    
                    try:
                        app_dlg.type_keys("{ENTER}")
                    except:
                        print('break here 1')
                        return None

                elif(i==0):
                    print('image_ke_{} classified as : SP2020_C1_RT'.format(i+1))
                    try:
                        list_items = combobox[0].children(control_type="ListItem")
                        if(list_items[0].window_text()=='SP2020_C1_ART'):
                            list_items[2].select()
                        else:
                            list_items[3].select()
                    except :
                        break
                    
                    try:
                        app_dlg.type_keys("{ENTER}")
                    except:
                        print('break here 2')
                        return None

                elif(i==1):
                    print('image_ke_{} classified as : SP2020_C1_LAMPIRAN'.format(i+1))
                    try:
                        list_items = combobox[0].children(control_type="ListItem")
                        if(list_items[0].window_text()=='SP2020_C1_ART'):
                            list_items[1].select()
                        else:
                            list_items[2].select()
                    except :
                        break
                    
                    try:
                        app_dlg.type_keys("{ENTER}")
                    except:
                        print('break here 3')
                        return None
                        
#                 sleep(2)
                try: 
                    app_dlg.type_keys("{ENTER}")
                except :
                    print('break here 4')
                    return None


                # check warning batch complete
                complete_dlg = app_dlg.descendants(title="All problems in the batch have been fixed. Close batch?")

                if(len(complete_dlg)>0):
                    # close batch
                    warning_close_batch_ok = app_dlg.descendants(title="OK", control_type="Button")
                    warning_close_batch_ok[0].type_keys('{ENTER}')
                    break

    else:
        for i in range(0, len(trees)):
            # identifikasi page dengan error
            error_images = len(trees[i].children()[0].children(class_name='ListBox')[0].children())
            
            if(error_images>0):
                combobox = trees[i].children()[0].children(class_name = 'ComboBox')
    
                # check combobox is visible
                trees[i].click_input()
                if(not combobox[0].children()[0].is_visible()):
                    combobox[0].click_input()
                
                # expand combobox
                combobox[0].children()[0].type_keys("%{DOWN}")
                sleep(1)

                # klasifikasi jenis dokumen
                if(i==0):
                    print('image_ke_{} classified as : SP2020_RC1'.format(i+1))
                    try:
                        list_items = combobox[0].children(control_type="ListItem")
                        if(list_items[0].window_text()=='SP2020_C1_ART'):
                            list_items[3].select()
                        else:
                            list_items[4].select()
                    except :
                        break
                    
                    try:
                        app_dlg.type_keys("{ENTER}")
                    except:
                        print('break here 5')
                        return None
                        

                elif(i==1):
                    print('image_ke_{} classified as : SP2020_C1_LAMPIRAN'.format(i+1))
                    try:
                        list_items = combobox[0].children(control_type="ListItem")
                        if(list_items[0].window_text()=='SP2020_C1_ART'):
                            list_items[1].select()
                        else:
                            list_items[2].select()
                    except :
                        break
                        
                    try:
                        app_dlg.type_keys("{ENTER}")
                    except:
                        print('break here 6')
                        return None
                        
#                 sleep(2)
                try: 
                    app_dlg.type_keys("{ENTER}")
                except :
                    print('break here 7')
                    return None


                # check warning batch complete
                complete_dlg = app_dlg.descendants(title="All problems in the batch have been fixed. Close batch?")

                if(len(complete_dlg)>0):
                    # close batch
                    warning_close_batch_ok = app_dlg.descendants(title="OK", control_type="Button")
                    warning_close_batch_ok[0].type_keys('{ENTER}')
                    break
            
            


In [ ]:
for i in tqdm(range(0,3000)):
    print('## batch_ke_{}'.format(i+1))
    auto_review_new()